In [1]:
import os
import open3d.ml as _ml3d
import open3d.ml.torch as ml3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
cfg_file = "ml3d/configs/pointpillars_kitti.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.PointPillars(**cfg.model)
cfg.dataset['dataset_path'] = "/workspace/data/kitti_detection/kitti"
dataset = ml3d.datasets.KITTI(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.ObjectDetection(model, dataset=dataset, device="gpu", **cfg.pipeline)


In [4]:
import torch
print(torch.cuda.is_available())
print(torch.__version__)


True
1.13.1+cu116


In [7]:
# download the weights.
ckpt_folder = "./logs/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "pointpillars_kitti_202012221652utc.pth"
pointpillar_url = "https://storage.googleapis.com/open3d-releases/model-zoo/pointpillars_kitti_202012221652utc.pth"
if not os.path.exists(ckpt_path):
    cmd = "wget {} -O {}".format(pointpillar_url, ckpt_path)
    os.system(cmd)


--2023-10-10 18:35:02--  https://storage.googleapis.com/open3d-releases/model-zoo/pointpillars_kitti_202012221652utc.pth
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.169.91, 216.58.201.123, 216.58.204.91, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.169.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58130305 (55M) [application/octet-stream]
Saving to: ‘./logs/pointpillars_kitti_202012221652utc.pth’

     0K .......... .......... .......... .......... ..........  0%  465K 2m2s
    50K .......... .......... .......... .......... ..........  0%  452K 2m4s
   100K .......... .......... .......... .......... ..........  0%  957K 1m42s
   150K .......... .......... .......... .......... ..........  0% 1.03M 90s
   200K .......... .......... .......... .......... ..........  0% 1.52M 79s
   250K .......... .......... .......... .......... ..........  0% 2.61M 69s
   300K .......... .......... .......... .......

In [8]:
# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

test_split = dataset.get_split("test")
data = test_split.get_data(0)


In [9]:
# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.
result = pipeline.run_inference(data)

/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [10]:
# evaluate performance on the test set; this will write logs to './logs'.
pipeline.run_test()

testing: 100%|██████████| 7518/7518 [06:05<00:00, 20.55it/s]
